# Python TD3

# Exercice 1

Question 1


In [1]:
ls -R toto

a     b     bla/  blu   c     titi/

toto/bla:
f  g

toto/titi:
d  e


In [2]:
import os
def listfiles(path):
    for root, dirs, files in os.walk(path):
        for f in files: print (os.path.join(root,f))

In [4]:
listfiles('toto')

toto/.DS_Store
toto/a
toto/blu
toto/c
toto/b
toto/bla/g
toto/bla/f
toto/titi/d
toto/titi/e


In [5]:
def count1(path):
    f=d=0
    for root, dirs, files in os.walk(path):
        f+=len(files); d+=len(dirs)
    print ("%d files, %d directories" % (f,d))

# ne compte pas "."

In [6]:
count1('toto')

9 files, 3 directories


In [7]:
def count2(path):
    f=s=d=0
    for root, dirs, files in os.walk(path):
        for dir_ in dirs:
            if os.path.islink(root+os.sep+dir_): s+=1
            else: d+=1
        for file_ in files:
            if os.path.islink(root+os.sep+file_): s+=1
            else: f+=1

    print ("%d directories, %d files, %d symlinks" % (d,f,s))

In [8]:
count2('toto')

3 directories, 9 files, 0 symlinks


In [10]:
# Un lien vers un fichier non existant peut générer une erreur
# On gère avec try/except
def stats(path):
    fpriv=fpub=spriv=spub=dpriv=dpub=0
    for root, dirs, files in os.walk(path):
        for dir_ in dirs:
            try:
                pub = os.stat(root+os.sep+dir_).st_mode % 64
                if os.path.islink(root+os.sep+dir_):
                    if pub: spub+=1
                    else: spriv+=1
                else:
                    if pub: dpub+=1
                    else: dpriv+=1
            except Exception as e :
                print (root+os.sep+dir_+' --> broken link!')
                print (e)
        for file_ in files:
            try:
                pub = os.stat(root+os.sep+file_).st_mode % 64
                if os.path.islink(root+os.sep+file_):
                    if pub: spub+=1
                    else: spriv+=1
                else:
                    if pub: fpub+=1
                    else: fpriv+=1
            except Exception as e:
                print (root+os.sep+file_ + ' --> broken link!')
                print (e)
    print()
    print ("Statistics of %s:" % path)
    print()
    print ("Public directories: %d" % dpub)
    print ("Private directories: %d" % dpriv)
    print ("Public files: %d" % fpub)
    print ("Private files: %d" % fpriv)
    print ("Public symlinks: %d" % spub)
    print ("Private symlinks: %d" % spriv)

In [11]:
stats('toto')



Statistics of toto:

Public directories: 3
Private directories: 0
Public files: 9
Private files: 0
Public symlinks: 0
Private symlinks: 0


In [12]:
ls -lR toto

total 0
-rw-r--r--  1 gogo  staff    0 Oct  8 16:26 a
-rw-r--r--  1 gogo  staff    0 Oct  8 16:26 b
drwxr-xr-x  4 gogo  staff  128 Oct  8 16:27 bla/
-rw-r--r--  1 gogo  staff    0 Oct  8 16:26 blu
-rw-r--r--  1 gogo  staff    0 Oct  8 16:26 c
drwxr-xr-x  5 gogo  staff  160 Nov  4 22:37 titi/

toto/bla:
total 0
-rw-r--r--  1 gogo  staff  0 Oct  8 16:27 f
-rw-r--r--  1 gogo  staff  0 Oct  8 16:27 g

toto/titi:
total 0
-rw-r--r--  1 gogo  staff  0 Oct  8 16:28 d
-rw-r--r--  1 gogo  staff  0 Oct  8 16:28 e


# Exercice 2

On parcourt récursivement l'arborescence du répertoire examiné, et on calcule un hachage (ici, md5) de chaque fichier rencontré. On utilise ces hachages (des chaines de 32 caractères hexadécimaux, par exemple 643385065da73cee1b42e504b0be1c8d) comme clefs d'un dictionnaire, dont les valeurs sont les listes des fichiers correspondant à ce hachage (donnés avec leur chemin absolu). On renvoie ensuite les listes ayant au moins deux éléments, et on les affiche au format demandé.

In [13]:
#!/usr/bin/env python3
'''
duplicates: lists identical files in a directory.

Usage: duplicates [-e] [--extension] [file extension],
                  [-d] [--directory] [directory]
                  [-o] [--outfile]   [output file]
                  |-h] [--help]
'''

import os, hashlib, sys, getopt


def find_duplicates(extension, directory):
    d = {}
    for root, dirs, files in os.walk(directory):
        for f in files:
            if not extension or os.path.splitext(f)[1][1:] == extension:
                p = os.path.join(root,f)
                try:
                    s = open(p,'rb').read()
                    md5 = hashlib.md5(s).hexdigest()
                    if md5 in d: d[md5].append(p)
                    else: d[md5] = [p]
                except: pass
    return d

def usage():
    'Print usage doc and exit.'
    print (__doc__)
    sys.exit()

if __name__ == '__main__':
    extension = None
    directory = os.getcwd() # portabilité : os.environ['PWD'] n'existe pas sous windows
    outfile = sys.stdout
    out = False

    try:
        optlist, arglist = getopt.getopt(sys.argv[1:],
                                         'he:d:o:',
                                         ['help', 'extension=',
                                          'directory=', 'outfile=' ])
    except:
        usage()
    for option in optlist:
        if option[0] in ['-h', '--help']: usage()
        if option[0] in ['-e', '--extension']: extension = option[1]
        if option[0] in ['-d', '--directory']: directory = option[1]
        if option[0] in ['-o', '--outfile']:
            outfile = open(option[1],'w')
            out = True
    d = find_duplicates(extension, directory)

    for x in d:
        if len(d[x])>1:
            for y in d[x]:
                outfile.write(y+'\n')
            outfile.write('\n')
    if out: outfile.close()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.




duplicates: lists identical files in a directory.

Usage: duplicates [-e] [--extension] [file extension],
                  [-d] [--directory] [directory]
                  [-o] [--outfile]   [output file]
                  |-h] [--help]

Traceback (most recent call last):
  File "/var/folders/3g/2pn9j21s1xj7ftm38ntxpt2m0000gn/T/ipykernel_3886/1402535156.py", line 40, in <module>
    optlist, arglist = getopt.getopt(sys.argv[1:],
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/getopt.py", line 95, in getopt
    opts, args = do_shorts(opts, args[0][1:], shortopts, args[1:])
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/getopt.py", line 195, in do_shorts
    if short_has_arg(opt, shortopts):
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/getopt.py", line 211, in short_has_arg
    raise 

TypeError: object of type 'NoneType' has no len()

# Exercice 3

On construit une expression régulière dist_pat(w) recherchant le remplacement ou effacement de la lettre à chaque position i possible, ou l'insertion d'une lettre avant la position i, en deux étapes.

In [16]:
import re

def dist_pat(w):
    # remplacement ou effacement de la lettre en position i
    ll = [ w[:i] + '.?' + w[i+1:] +'$'  for i in range(len(w))]
    # insertion d'une lettre avant la position i
    ll.extend([w[:i] + '.' + w[i:] + '$' for i in range(len(w)+1)])
    # le "ou" de toutes ces conditions
    return '(' + '|'.join(ll) + ')'


def lookup(w):
    ll = open('dictionaire.txt', encoding='latin1').readlines()
    if w:
        s=dist_pat(w); p=re.compile(s, re.U)
        return [w.rstrip() for w in ll if re.match(p,w)]
    else: return [x.rstrip() for x in ll if len(x.rstrip())==1]

In [17]:
print (lookup('ais'))

['agis', 'ai', 'aie', 'aies', 'ail', 'ails', 'air', 'airs', 'ais', 'aise', 'aisy', 'ait', 'amis', 'anis', 'ans', 'apis', 'ars', 'as', 'avis', 'axis', 'bais', 'bis', 'dais', 'dis', 'fais', 'fis', 'gais', 'gis', 'hais', 'jais', 'lais', 'lis', 'mais', 'mis', 'nais', 'ois', 'pais', 'pis', 'rais', 'ris', 'sais', 'sis', 'tais', 'vais', 'vis']


In [18]:
dist_pat('ais')

'(.?is$|a.?s$|ai.?$|.ais$|a.is$|ai.s$|ais.$)'